In [1]:
import geopandas as gpd
import urban_connector as uc

# Area of interest (AOI)

In [2]:
# load AOI polygon
gdf_poly = gpd.read_file("./ex_data/bs_as.geojson")

# Graph

In [3]:
# Create graph
G = uc.osmnx_graph_from_polygon(gdf_poly, 'walk', 4.5)

In [4]:
# Create network nodes and lines
gdf_nodes, gdf_edges, dict_mapping_nodes = uc.create_geodata_from_graph(G)

In [5]:
# Export nodes and lines
# gdf_nodes.to_file('./ex_data/nodes_all.shp')
# gdf_edges[['osmid', 'reversed', 'node_id_st', 'node_id_en', 'geometry']].to_file('./ex_data/edges_all.shp')

# Sources

In [7]:
# Set amenities to download and the boundary polygon
amenities = ["hospital"] # "pharmacy", "hospital", "school"
boundary_polygon = gdf_poly.loc[0, "geometry"]

In [8]:
# Download amenities from OSM
gdf_amenities = uc.get_amenities(boundary_polygon, amenities)
print(len(gdf_amenities))

108


d:\GIT\urban_connector\urban_connector\manager\sources_manager.py:30: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf_amenities = ox.geometries_from_polygon(poly_boundary, tags={"amenity": amenities}).loc[:, ["name", "amenity", "geometry"]].reset_index(drop=True)
d:\GIT\urban_connector\urban_connector\manager\sources_manager.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_amenities.geometry = gdf_amenities.geometry.centroid


In [9]:
# Find all nodes closest to sources
list_node_sources = uc.closest_nodes_to_sources(G, gdf_amenities)

In [6]:
# list of nodes by buffer
# list_node_sources_by_buffer = search_node_near_to_sources (gdf_amenities, gdf_nodes, search_radio=50)

# Isochrones

In [19]:
# Set time limits
list_times = [5, 10, 15]

In [20]:
# Create isochrones, rings and edges with travel times
# long function - output: edges, isochrones, rings
list_costs_buffers = []
for v_cost_limit in list_times:
    df_edges_cost_limit = uc.calculate_service_edges(G, gdf_nodes, gdf_edges, 'weight', v_cost_limit, list_node_sources)
    df_edges_cost_limit = df_edges_cost_limit[['node_id_st', 'node_id_en', 'weight', 'w_start', 'w_end', 'w_mean', 'w_limit', 'geometry']]
    df_edges_cost_limit.to_file(f"./ex_data/edges_weight_{v_cost_limit}.shp")
    gdf_buffer = uc.create_service_area(df_edges_cost_limit, "3857", 5, 100)
    gdf_buffer.to_file(f"./ex_data/isochrones_weight_{v_cost_limit}.shp")
    list_costs_buffers.append(gdf_buffer)

gdf_service_areas = uc.merge_service_areas_polygons( list_costs_buffers, 'rings')
gdf_service_areas.to_file("./ex_data/isochrones_rings.shp")

c:\Users\Compumar\Documents\PythonEnvs\AppCEEU\Lib\site-packages\geopandas\geodataframe.py:2451: UserWarning: `keep_geom_type=True` in overlay resulted in 80 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(


In [ ]:
# Create isochrones, rings and edges with travel times
# short function - output: only rings
""" gdf_service_areas_multiple_cost = uc.create_service_area_for_multiple_weights(G, gdf_nodes, gdf_edges, 'weight', [5, 10, 15], list_node_sources, 
                                       5, 100, 'rings')
gdf_service_areas_multiple_cost.to_file("./ex_data/isochrones.shp") """

# ADD NEW SOURCE

In [10]:
# Add a new source
gdf_new_sources = gpd.read_file("./ex_data/new_source.shp")
gdf_new_sources = uc.build_source_gdf(gdf_new_sources, field_category='categoria', field_name="nombre")
gdf_new_sources, list_node_sources_new = uc.add_amenities(gdf_new_sources, gdf_amenities, list_node_sources, G, str_method_near_nodes=['closest'])
gdf_new_sources

,src_id,category,name,geometry
0,src_1,hospital,Sanatorio De La Providencia,POINT (-58.39366 -34.60195)
1,src_2,hospital,Sanatorio Anchorena,POINT (-58.40030 -34.58922)
2,src_3,hospital,Instituto Sacre Coeur,POINT (-58.41165 -34.59528)
3,src_4,hospital,Sanatorio Franchín,POINT (-58.41669 -34.60938)
4,src_5,hospital,Centro de Diagnóstico Parque,POINT (-58.49761 -34.60095)
...,...,...,...,...
104,src_105,hospital,Climedica,POINT (-58.40183 -34.62162)
105,src_106,hospital,Hospital Aeronáutico Central,POINT (-58.41233 -34.65281)
106,src_107,hospital,Hospital Italiano de Buenos Aires,POINT (-58.42365 -34.60784)
107,src_108,hospital,Hospital Militar Central Cirujano Mayor Doctor...,POINT (-58.43722 -34.57053)


In [11]:
# Set time limits
list_times = [5, 10, 15]

# Create isochrones, rings and edges with travel times
# long function - output: edges, isochrones, rings
list_costs_buffers = []
for v_cost_limit in [5, 10, 15]:
    df_edges_cost_limit = uc.calculate_service_edges(G, gdf_nodes, gdf_edges, 'weight', v_cost_limit, list_node_sources_new)
    df_edges_cost_limit = df_edges_cost_limit[['node_id_st', 'node_id_en', 'weight', 'w_start', 'w_end', 'w_mean', 'w_limit', 'geometry']]
    df_edges_cost_limit.to_file(f"./ex_data/new_edges_weight_{v_cost_limit}.shp")
    gdf_buffer = uc.create_service_area(df_edges_cost_limit, "3857", 5, 100)
    gdf_buffer.to_file(f"./ex_data/new_isochrones_weight_{v_cost_limit}.shp")
    list_costs_buffers.append(gdf_buffer)

gdf_service_areas = uc.merge_service_areas_polygons( list_costs_buffers, 'rings')
gdf_service_areas.to_file("./ex_data/new_isochrones_rings.shp")

c:\Users\Compumar\Documents\PythonEnvs\AppCEEU\Lib\site-packages\geopandas\geodataframe.py:2451: UserWarning: `keep_geom_type=True` in overlay resulted in 93 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return geopandas.overlay(
